<a href="https://colab.research.google.com/github/nikashahabi/Cryptic-Unstable-Transcripts-Identification-with-Deep-Learning/blob/main/Part5_AddingNewDataAndGettingItsEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 snRNA_df = pd.read_csv ('/content/drive/MyDrive/Bioinformatics/snRNA.csv', header = None)
 snoRNA_df = pd.read_csv ('/content/drive/MyDrive/Bioinformatics/snoRNA.csv', header = None)

In [ ]:
snRNA_df

,0
0,GTTCGCGAAGTAACCCTTCGTGGACATTTGGTCAATTTGAAACAAT...
1,ATACTTACCTTAAGATATCAGAGGAGATCAAGAAGTCCTACTGATC...
2,ACGAATCTCTTTGCCTTTTGGCTTAGATCAAGTGTAGTATCTGTTC...
3,ATCCTTATGCACGGGAAATACGCATATCAGTGAGGATTCGTCCGAG...
4,AAGCAGCTTTACAGATCAATGGCGGAGGGAGGTCAACATCAAGAAC...
5,AAGCAGCTTTACAGATCAATGGCGGAGGGAGGTCAACATCAAGAAC...


In [ ]:
!pip install -qU sentence-transformers
!pip install sentencepiece

     |████████████████████████████████| 79 kB 3.2 MB/s 
     |████████████████████████████████| 4.2 MB 41.3 MB/s 
     |████████████████████████████████| 1.2 MB 56.8 MB/s 
     |████████████████████████████████| 86 kB 3.9 MB/s 
     |████████████████████████████████| 6.6 MB 52.0 MB/s 
     |████████████████████████████████| 596 kB 66.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import models, SentenceTransformer, util
from transformers import BertTokenizer,BertModel
model = BertModel.from_pretrained('/content/drive/MyDrive/Bioinformatics/3-new-12w-0.zip (Unzipped Files)/3-new-12w-0')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Bioinformatics/3-new-12w-0.zip (Unzipped Files)/3-new-12w-0')

def get_kmer_sentence(original_string, kmer = 1, stride = 1):
  if kmer == -1:
    return original_string
  sentence = ""
  original_string = original_string.replace("\n", "")
  i = 0
  while i < len(original_string)-kmer:
      sentence += original_string[i:i+kmer] + " "
      i += stride
  return sentence[:-1].strip("\"")

def get_kmer_sequence(original_string, kmer = 1):
  if kmer == -1:
    return original_string

  sequence = []
  original_string = original_string.replace("\n", "")
  for i in range(len(original_string)-kmer):
    sequence.append(original_string[i:i+kmer])
    
  sequence.append(original_string[-kmer:])
  return sequence
def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)
    
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model
embedder = load_st_model('/content/drive/MyDrive/Bioinformatics/3-new-12w-0.zip (Unzipped Files)/3-new-12w-0')
def get_vector(sequence):
  if len(sequence) > 512:
    print("ERROR: Length > 512")
    return None
  else:
    three_mer_sequence = get_kmer_sentence(sequence, kmer = 3)
    "az in ja codeE ke sobhan vared nakarde bud"
    # embedder = load_st_model("Rostlab/prot_bert")
    vector_idk = embedder.encode(three_mer_sequence, convert_to_tensor = True, show_progress_bar = True )
    print("chizi ke man estefade kardam:", np.array(vector_idk.tolist()))
    print(np.array(vector_idk.tolist()).shape)
    print("chizi ke comment bud estefade nakardam")
    # in ghesmat code comment shodas ke sobhan bardashte budesh. mide (len-1 , 768) vali code jadid 1024 taE mide baraye ye sequence
    vector = model(**tokenizer(three_mer_sequence, return_tensors = 'pt'))[0].detach().squeeze()
    vector = vector.tolist()
    vector = np.array(vector)
    print(vector.shape)
    print(vector)
    return np.array(vector_idk.tolist())

Some weights of the model checkpoint at /content/drive/MyDrive/Bioinformatics/3-new-12w-0.zip (Unzipped Files)/3-new-12w-0 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /content/drive/MyDrive/Bioinformatics/3-new-12w-0.zip (Unzipped Files)/3-new-1

### Getting vectors of each sequence

In [ ]:
def give_vector(row): 
    return get_vector(row['sequence'])
  


In [ ]:
snRNA_df.columns = ['sequence']
snoRNA_df.columns = ['sequence']
snRNA_df

,sequence
0,GTTCGCGAAGTAACCCTTCGTGGACATTTGGTCAATTTGAAACAAT...
1,ATACTTACCTTAAGATATCAGAGGAGATCAAGAAGTCCTACTGATC...
2,ACGAATCTCTTTGCCTTTTGGCTTAGATCAAGTGTAGTATCTGTTC...
3,ATCCTTATGCACGGGAAATACGCATATCAGTGAGGATTCGTCCGAG...
4,AAGCAGCTTTACAGATCAATGGCGGAGGGAGGTCAACATCAAGAAC...
5,AAGCAGCTTTACAGATCAATGGCGGAGGGAGGTCAACATCAAGAAC...


In [ ]:

snRNA_df['vectors'] = snRNA_df.apply(lambda row: give_vector(row), axis=1)
snoRNA_df['vectors'] = snoRNA_df.apply(lambda row: give_vector(row), axis=1)
snRNA_df["label"] = 0
snoRNA_df["label"] = 0
snRNA_df.to_csv("snRNA_sq_with_vectors")
snoRNA_df.to_csv("snoRNA_sq_with_vectors")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

chizi ke man estefade kardam: [-0.02888303 -0.06926857  0.84890974  0.67760694  0.31778806 -1.01144195
 -0.05326768 -0.04660594 -0.30503014  0.20600368  0.13509361 -0.52825481
  0.24372934  0.17904708 -0.06568143  0.37028995 -0.45866704 -0.03254639
  0.78084224 -0.2362946  -0.20209613 -0.68870813 -0.48908812 -0.36222148
 -0.49450275  0.02753815  0.46382275  0.64235187  0.14253023  0.00813842
 -0.23999909 -0.29300961  0.02707837  0.1586048  -0.7595517   0.32355684
  0.25284088 -0.51728231 -0.05710447  0.55635923 -0.36623135 -0.46649507
 -0.19552189 -0.34278783 -0.02811232  0.602934    0.12203655  0.65744573
  0.28971228 -0.12397555 -0.16856141  0.54228449  0.29098532  0.47211626
  0.44754991  0.45841986  0.47014347  0.83845639 -0.16297802  0.3571246
 -0.38910168 -0.31243491 -0.8292743  -0.22489566 -0.14428401 -0.15769158
  0.83116382 -0.32744783  0.66166991 -0.7992419   0.1499818   0.15213963
 -0.67639035 -0.24791239  0.15971012  0.62222558 -0.41850972  0.34956634
 -0.37055102 -0.635000

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

chizi ke man estefade kardam: [ 3.83805856e-02  1.19332513e-02  7.68192232e-01  4.80853468e-01
  2.59636432e-01 -9.62424219e-01  5.37749603e-02 -3.19040984e-01
 -3.40432882e-01  2.61926234e-01  1.40345961e-01 -6.60799801e-01
  3.55320096e-01  3.48582596e-01 -2.58753151e-01  4.48102176e-01
 -4.17001009e-01  2.18091384e-01  7.38851488e-01 -3.03588659e-01
 -3.54276478e-01 -7.74621665e-01 -4.63646799e-01 -2.11383417e-01
 -3.38960499e-01 -3.02526187e-02  3.16735625e-01  6.76448762e-01
  8.44787061e-02 -2.57861847e-03 -3.12767476e-01 -3.18202257e-01
  1.13186389e-01  1.32600158e-01 -8.54363620e-01  2.56936193e-01
  3.35299760e-01 -2.73882121e-01 -7.96191171e-02  4.78077620e-01
 -4.42768931e-01 -5.32906830e-01 -1.34298056e-01 -3.79038215e-01
  6.11825138e-02  7.03216553e-01  1.17277183e-01  7.41961718e-01
  3.28466356e-01 -2.02748135e-01 -7.55869001e-02  5.78480840e-01
  3.37144732e-01  4.31553006e-01  5.73258579e-01  4.29035127e-01
  4.69724655e-01  8.42921972e-01 -1.85864016e-01  3.21286857

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

chizi ke man estefade kardam: [-2.31410518e-01  1.34690598e-01  5.58000445e-01  1.88921213e-01
  1.64324701e-01 -1.08848238e+00 -1.02415020e-02 -2.47477472e-01
 -1.39842868e-01  3.83813620e-01  1.05094090e-01 -3.70271444e-01
  2.03207389e-01  1.45109594e-01 -4.69265804e-02  1.47005618e-01
 -3.34888548e-01  3.05349469e-01  6.97713256e-01 -3.33972812e-01
 -2.33622625e-01 -6.07481539e-01 -4.92793560e-01 -7.98571780e-02
 -3.04713368e-01 -1.55621236e-02  4.25092846e-01  1.36714011e-01
 -1.16488725e-01 -6.35577962e-02 -4.46092099e-01 -2.22554535e-01
  2.23832443e-01  2.94340849e-01 -4.13559765e-01  1.70255199e-01
  5.20064473e-01 -8.42605755e-02 -2.56512109e-02  4.73608047e-01
 -6.70968592e-01 -4.08829659e-01  9.50448364e-02 -1.91518933e-01
 -1.25051185e-03  4.27118242e-01  1.60058707e-01  5.11136770e-01
  3.11135918e-01 -1.96150586e-01  1.93431020e-01  3.36228251e-01
  1.79349750e-01  4.05596077e-01  6.10409975e-01  5.85512757e-01
  5.44602454e-01  7.79091358e-01 -1.48690850e-01  2.47255951

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

chizi ke man estefade kardam: [-2.94804275e-01  8.30451101e-02 -3.68141830e-01 -2.10564047e-01
 -2.49986753e-01 -1.64193106e+00 -1.72284186e-01 -1.83410659e-01
 -4.20179784e-01  2.03290254e-01  3.11989695e-01  6.77656010e-02
  1.37885157e-02  1.82181790e-01 -9.91379246e-02 -1.87369570e-01
 -1.87024474e-01  2.36711860e-01  2.51299322e-01 -2.54683614e-01
 -1.81704700e-01  9.32012498e-02 -6.52620912e-01 -1.56692520e-01
 -1.33124694e-01  1.75253555e-01  2.36712888e-01 -1.82789981e-01
  1.17761418e-01 -6.41816035e-02 -2.74598241e-01  1.15728326e-01
  6.97928062e-03  5.74544191e-01 -1.93380713e-01  7.05250055e-02
  3.26804966e-01  2.35958840e-03 -4.86237966e-02  6.75364614e-01
 -8.73697698e-01 -1.20686844e-01  1.26820520e-01  2.33577743e-01
 -1.46532312e-01 -3.73410434e-01  4.18822885e-01  2.10831076e-01
  5.60266256e-01 -1.42539376e-02  8.08758914e-01  3.20752829e-01
  3.04763615e-01  4.55888093e-01  3.40012848e-01  4.01499301e-01
  6.52109563e-01  5.55925429e-01 -8.42519179e-02  1.43671587

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

chizi ke man estefade kardam: [-5.72369322e-02  1.54329434e-01  2.50229519e-02  2.57493526e-01
  2.13011831e-01 -7.03505397e-01  3.69797111e-03 -2.18062207e-01
 -2.45112419e-01  4.45604026e-01  1.41161650e-01 -1.73078209e-01
  2.47422934e-01 -4.89714183e-02  2.28748038e-01  2.20238604e-02
 -3.14945281e-01  1.72116905e-01  6.65988922e-01 -2.40422532e-01
 -2.00220764e-01 -6.08076096e-01 -4.90195483e-01 -3.97378296e-01
 -5.27707487e-02  2.34085485e-01  4.97553229e-01  1.74108922e-01
 -1.01229250e-02 -3.24930623e-02 -3.56330574e-01 -5.16536653e-01
  2.19870374e-01  3.10806394e-01 -4.64794338e-01 -2.29292805e-03
  6.44329906e-01  2.27628782e-01 -1.32495403e-01  3.88071656e-01
 -7.91834652e-01 -6.00060940e-01  8.67143795e-02 -1.55453086e-01
  4.90771122e-02  2.56313682e-01  2.50667393e-01  4.92185801e-01
  2.69605994e-01 -2.41222501e-01 -1.79907363e-02  3.68896931e-01
  9.20790955e-02  3.11935604e-01  3.72706354e-01  3.78010482e-01
  4.56656218e-01  6.80431545e-01 -2.64420081e-02  1.56085432

In [ ]:
df_before_shuffled = pd.read_pickle("/content/drive/MyDrive/Bioinformatics/shuffled_sq_with_labels_and_encodings.pkl")
snoRNA_df

,sequence,vectors,label
0,AACTTTGTCCTAAAGTACTAATCCACCGCATTAGACAGTACGAAGA...,"[0.2477642148733139, -0.2145836353302002, 0.99...",0
1,AACATCAGTAGTGGTTAATACTATCTTTAGATAATATTAATTTCAT...,"[-0.02805790677666664, -0.10658550262451172, 0...",0
2,TGTTAATGATGATAACCAAAGATGCATAGTTCAACTGATTGAACAT...,"[-0.046499334275722504, 0.040201060473918915, ...",0
3,GGCCCTGATGATAATGGTGTCTCTTCTTTCCTCGTCCGATTCGACC...,"[-0.3696243166923523, 0.04711676016449928, 0.7...",0
4,TCACGGTGATGAAAGACTGGTTCCTTAACATTCGCAGTTTCCACGG...,"[0.3796132504940033, -0.11162212491035461, 0.7...",0
...,...,...,...
72,TCATAATGATGATTTCACTTATACCTATATGTTTTTTCTGGCATCT...,"[-0.2748427391052246, 0.2322671115398407, -0.2...",0
73,ATGGCCTCTTCGAAGATCCGTAGATTTTGCATTTTTGCGGTGGTTT...,"[-0.36765825748443604, 0.01476874016225338, 0....",0
74,AACATCATAATTATATCGCGAATGTACTACTAGTATATGCAGTTTT...,"[-0.23141051828861237, 0.13469059765338898, 0....",0
75,GTTATATGATGATAACCTTCTCAGCTCACTCAGATCTTTTGATATG...,"[-0.29480427503585815, 0.0830451101064682, -0....",0


In [ ]:
all_sq_df_new = pd.concat([df_before_shuffled, snoRNA_df, snRNA_df])
shuffled_df_new = all_sq_df_new.sample(frac = 1)

In [ ]:
shuffled_df_new.to_pickle("/content/drive/MyDrive/Bioinformatics/shuffled_df_new_with_sequences_vectors_and_labels.pkl")